In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import random

import torch.optim as optim
from torch.optim import lr_scheduler

from collections import OrderedDict
from tqdm import tqdm

from torchsummary import summary

from albumentations.augmentations import transforms, Normalize
from albumentations import Flip, RandomRotate90, Resize
from albumentations.core.composition import Compose, OneOf

from sklearn.model_selection import train_test_split

import argparse

import models
import metrics
import utils
from train import *
from data import BUSIDataset

config = argparse.Namespace()

config.model = 'UNet2Plus'

config.SIZE = 128
config.batch_size = 8
config.num_workers = 8
config.n_channels = 1
config.lr = 0.001
config.min_lr = 0.00001
config.epochs = 200
config.early_stopping = 50
config.patience = config.early_stopping
config.base_dir = ''
config.root_path = os.path.join(config.base_dir, 'breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/')
config.semantic = False
config.device = torch.device('cuda:0')
config.classes = ['normal', 'benign', 'malignant']
config.labels = []
config.num_classes = 3 if config.semantic else 1
config.loss = 'BCEDiceLoss'
config.optimizer = 'Adam'
config.scheduler = 'CosineAnnealingLR'
config.weight_decay = 1e-4
config.momentum = 0.9
config.nesterov = False
config.seed = 4

if config.semantic:
    config.labels = config.classes
else:
    config.labels = ['cancer']

# write a function that sets the seed of all the libraries
def set_seed():
    torch.manual_seed(config.seed)
    np.random.seed(config.seed)
    random.seed(config.seed)

In [5]:
def run_model(config):
    if config.model is not None and not os.path.exists('model_files/%s' % config.model):
        os.makedirs('model_files/%s' % config.model)

    print('-' * 20)
    for key, value in config.__dict__.items():
        print('%s: %s' % (key, value))
    print('-' * 20)


    criterion = metrics.__dict__[config.loss]().cuda()
    model = models.__dict__[config.model](n_channels=config.n_channels,
                                            n_classes=config.num_classes).cuda()


    summary(model, (config.n_channels, config.SIZE, config.SIZE))

    params = filter(lambda p: p.requires_grad, model.parameters())
    num_params = sum([p.numel() for p in model.parameters()])

    if config.optimizer == 'Adam':
        optimizer = optim.Adam(
            params, lr=config.lr, weight_decay=config.weight_decay)
    elif config.optimizer == 'SGD':
        optimizer = optim.SGD(params, lr=config.lr, momentum=config.momentum,
                                nesterov=config.nesterov, weight_decay=config.weight_decay)

    if config.scheduler == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=config.epochs, eta_min=config.min_lr)
    elif config.scheduler == 'ReduceLROnPlateau':
        scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=config.factor, patience=config.patience,
                                                    verbose=1, min_lr=config.min_lr)
    else:
        scheduler = None

    image_paths, mask_paths = utils.load_dataset(config.root_path)
    print(f"Total Images: {len(image_paths)}, Masks: {len(mask_paths)}")

    train_transform = Compose([
        RandomRotate90(),
        Flip(),
    ])

    # image_paths, mask_paths = image_paths[:20], mask_paths[:20] # overfitting
    X_train, X_test, y_train, y_test = train_test_split(image_paths, mask_paths,  test_size=0.4, random_state=config.seed)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=config.seed)
    # split X_test, y_test into validation and test, 50-50

    train_dataset = BUSIDataset(image_paths=X_train, mask_paths=y_train, size=config.SIZE, transform=train_transform)
    val_dataset = BUSIDataset(image_paths=X_val, mask_paths=y_val, size=config.SIZE, transform=None)
    test_dataset = BUSIDataset(image_paths=X_test, mask_paths=y_test, size=config.SIZE, transform=None)

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.batch_size,
        shuffle=True,
        drop_last=False)

    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=config.batch_size,
        shuffle=True,
        drop_last=False)

    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=config.batch_size,
        shuffle=True,
        drop_last=False)

    log = OrderedDict([
        ('train_lr', []),
        ('train_loss', []),
        ('train_iou', []),
        ('train_dice', []),
        ('val_loss', []),
        ('val_iou', []),
        ('val_dice', []),
    ])

    best_dice = 0
    trigger = 0
    total_trained_epochs = 0
    for epoch in range(config.epochs):
        print('Epoch [%d/%d]' % (epoch, config.epochs))
        total_trained_epochs += 1

        # train for one epoch
        train_log = train(train_loader, model, criterion, optimizer)
        # evaluate on validation set
        val_log = validate(val_loader, model, criterion)

        if config.scheduler == 'CosineAnnealingLR':
            scheduler.step()
        elif config.scheduler == 'ReduceLROnPlateau':
            scheduler.step(val_log['loss'])

        print(f"loss {train_log['loss']:.4f} - iou {train_log['iou']:.4f} - dice {train_log['dice']:.4f} \
                - val_loss {val_log['loss']:.4f} - val_iou {val_log['iou']:.4f} - val_dice {val_log['dice']:.4f}") 

        print(train_log, val_log)
        log['train_lr'].append(config.lr)
        log['train_loss'].append(train_log['loss'])
        log['train_iou'].append(train_log['iou'])
        log['train_dice'].append(train_log['dice'])
        log['val_loss'].append(val_log['loss'])
        log['val_iou'].append(val_log['iou'])
        log['val_dice'].append(val_log['dice'])

        pd.DataFrame(log).to_csv('model_files/%s/log.csv' %
                                    config.model, index=False)

        trigger += 1

        if val_log['dice'] > best_dice:
            torch.save(model.state_dict(), 'model_files/%s/model.pth' %
                        config.model)
            best_dice = val_log['dice']
            print("=> saved best model")
            trigger = 0

        # early stopping
        if config.early_stopping >= 0 and trigger >= config.early_stopping:
            print("=> early stopping")
            break

        torch.cuda.empty_cache()

    print(f"Val Loss: {log['val_loss'][-1]:.4f} - Val IOU: {log['val_iou'][-1]:.4f} - Val Dice: {log['val_dice'][-1]:.4f}")

    model.load_state_dict(torch.load('model_files/%s/model.pth' % config.model))
    test_log = validate(test_loader, model, criterion)
    print(f"Test loss {test_log['loss']:.4f} - Test IoU {test_log['iou']:.4f} - Test Dice {test_log['dice']:.4f}")
    test_log_df = pd.DataFrame({'model': [config.model], 'epochs': [total_trained_epochs], 'size': [config.SIZE], 'params': [num_params], 'test_dice': [test_log['dice']], 'test_iou': [test_log['iou']], 'test_loss': [test_log['loss']], 'seed' : [config.seed]})
    # append test_log_df to test_log.csv
    test_log_df.to_csv('test_log.csv', mode='a', header=False, index=False)

In [6]:
# set seeds of 4, 9, 17, 18, 38
seeds = [4, 9, 17, 18, 38]
model_names = ['UNet2Plus', 'UNet3Plus']

for seed in seeds:
    for model_name in model_names:
        config.seed = seed
        config.model = model_name
        set_seed()
        run_model(config)

--------------------
model: UNet3Plus
SIZE: 128
batch_size: 8
num_workers: 8
n_channels: 1
lr: 0.01
min_lr: 1e-05
epochs: 200
early_stopping: 50
patience: 50
base_dir: 
root_path: breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/
semantic: False
device: cuda:0
classes: ['normal', 'benign', 'malignant']
labels: ['cancer']
num_classes: 1
loss: BCEDiceLoss
optimizer: Adam
scheduler: CosineAnnealingLR
weight_decay: 0.0001
momentum: 0.9
nesterov: False
seed: 4
--------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─unetConv2: 1-1                         [-1, 64, 128, 128]        --
|    └─Sequential: 2-1                   [-1, 64, 128, 128]        --
|    |    └─Conv2d: 3-1                  [-1, 64, 128, 128]        640
|    |    └─BatchNorm2d: 3-2             [-1, 64, 128, 128]        128
|    |    └─ReLU: 3-3                    [-1, 64, 128, 128]        --
|    └─Sequential: 2-2                   [-1, 64, 128, 128]        --
|    |    └─Conv2

100%|██████████| 17/17 [00:04<00:00,  4.20it/s, loss=1.22, iou=1.32e-9, dice=2.64e-9]


loss 1.1980 - iou 0.0167 - dice 0.0317                 - val_loss 1.2157 - val_iou 0.0000 - val_dice 0.0000
OrderedDict({'loss': 1.1979777481138092, 'iou': 0.01666131311103386, 'dice': 0.03172044034570515}) OrderedDict({'loss': 1.2156619064567624, 'iou': 1.3222164727995258e-09, 'dice': 2.644432919682559e-09})
=> saved best model
Epoch [1/200]


100%|██████████| 17/17 [00:04<00:00,  4.19it/s, loss=1.22, iou=0.000249, dice=0.000498]


loss 1.1964 - iou 0.0053 - dice 0.0105                 - val_loss 1.2157 - val_iou 0.0002 - val_dice 0.0005
OrderedDict({'loss': 1.1963835340185263, 'iou': 0.0053455014605466155, 'dice': 0.01047078021874723}) OrderedDict({'loss': 1.2156618972157323, 'iou': 0.00024924624612654504, 'dice': 0.000497775662818066})
=> saved best model
Epoch [2/200]


KeyboardInterrupt: 